In [160]:
import strawberry
from typing import Any, List, Dict, Type
from strawberry.directive import DirectiveLocation
from strawberry.types import Info
import regex

# Define a custom directive
@strawberry.directive(locations=[DirectiveLocation.FIELD], description="Make string uppercase")
def uppercase(value: str):
    return value.upper()

# Define a custom directive
@strawberry.directive(locations=[DirectiveLocation.FIELD], description="Make string lowercase")
def lower(value: str):
    return value.lower()

# Define a custom directive
@strawberry.directive(locations=[DirectiveLocation.FIELD], description="Matches")
def extract(value: str, expression: str):
    result = regex.match(expression, value)
    if result is not None:
        return result.group(0)
    return ""


def dynamic_class(name: str, annotations: dict):
    class_type = type(name, (), {})
    class_type.__annotations__ = annotations
    return strawberry.type(class_type)


User = dynamic_class('User', {'name': str, 'age': int})

@strawberry.type
class Query:
    @strawberry.field
    def user(self) -> List["User"]:
        return [User(name="Patrick", age=100), User(name="Mary", age=100)]

schema = strawberry.Schema(query=Query, directives=[uppercase, lower, extract]) 

query = """
{
    user {
        name @extract(expression:"P.+") @uppercase
        age
    }
}
"""

result = schema.execute_sync(query)
print(result)

In [122]:
# program to create class dynamically 
  
# constructor 
def constructor(self, arg): 
    self.constructor_arg = arg 
  
# method 
def displayMethod(self, arg): 
    print(arg) 
  
# class method 
@classmethod
def classMethod(cls, arg): 
    print(arg) 
  
# creating class dynamically 
Geeks = type("Geeks", (object, ), { 

    # data members 
    "string_attribute": "Geeks 4 geeks !", 
    "int_attribute": 1706256, 
      
    # member functions 
    "func_arg": displayMethod, 
    "class_func": classMethod
}) 

vars(strawberry.type(Geeks)())

In [150]:
class X:
    x: int

X = type('X', (), {})

X.__annotations__ = {'x': int}


strawberry.type(X)(x=0)

In [180]:
import duckdb

con = duckdb.connect("file.db")
```
CREATE TABLE items (
    id INTEGER PRIMARY KEY,
    value TEXT
)
''')


In [45]:
import duckdb
from duckdb.typing import *

def generate_random_name():
    return "1"

con.create_function("random_name", generate_random_name, [], VARCHAR)

In [1]:
import duckdb
import asyncio
import random
import string

# Create a DuckDB in-memory database
conn = duckdb.connect(':memory:')

# Create a table with three keys
conn.execute('''
CREATE TABLE items (
    id1 INTEGER,
    id2 INTEGER,
    id3 INTEGER,
    value TEXT,
    PRIMARY KEY (id1, id2, id3)
)
''')

# Function to generate random string
def random_string(length=10):
    return ''.join(random.choices(string.ascii_lowercase, k=length))

# Insert random data
for i in range(1, 11):
    for j in range(1, 4):
        for k in range(1, 3):
            conn.execute('''
            INSERT INTO items (id1, id2, id3, value) VALUES (?, ?, ?, ?)
            ''', (i, j, k, random_string()))

# Define an asynchronous function to simulate an external API call
async def async_fetch_data(value: str) -> str:
    return f"processed_{value}"

# Define a synchronous wrapper for the async function
def fetch_data(value: str) -> str:
    # Run the async function and wait for the result
    result = asyncio.run(async_fetch_data(value))
    return result

# Register the UDF with DuckDB
conn.create_function("fetch_data", fetch_data, [str], str)

# Execute a query that uses the custom UDF
result = conn.execute('''
SELECT id1, id2, id3, fetch_data(value) as processed_value FROM items
''').fetchall()

# Print the results
for row in result:
    print(row)
